In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
# pd.set_option('display.max_rows', 10000)
# pd.set_option('display.max_columns', 10000)

* 資料清理


In [ ]:
def fill(df): # 填充缺值
  for i in range(0, 1650): # 1650 行需特殊處理，因此停在這
    for j in range(2, len(df.columns)):
      if df.iloc[i, j] == -1:
        df.iloc[i, j] = (find_value(df, i, j, 0) + find_value(df, i, j, 1))/2
  

In [ ]:
def find_value(df, x, y, z): # x is row, y is col, z is finding left or right [0:left, 1:right]
  if(z == 0):
    for i in range(y-1, 1, -1): # 往前推算
      if df.iloc[x, i] != -1:
        # print(x, i) # 確認找到的index與vlaue
        return df.iloc[x, i]
    return find_value(df, x-18, len(df.columns), 0)
  
  else:
    for i in range(y+1, len(df.columns)): # 往後推算
      if df.iloc[x, i] != -1:
        # print(x, i) 
        return df.iloc[x, i]
    return find_value(df, x+18, 1, 1)

In [ ]:
def split_dataset(df, df_y, hr): # hr為幾小時, dy_f為test set
  # 切割x_train部分
  x_train_all = []
  x_train_pm25 = []

  # 切割全部的feature為x
  for i in range(0, df.shape[0]-hr):
    for j in range(i, i+6):
      x_train_all = np.concatenate((x_train_all, np.array(df.iloc[j, :])), axis=0)
  
  # 切割全部的pm2.5為x
  for i in range(0, df.shape[0]-hr):
    for j in range(i, i+6):
      x_train_pm25 = np.concatenate((x_train_pm25, [np.array(df.iloc[j, 9])]), axis=0)
  
  print(x_train_all.shape)
  print(x_train_pm25.shape)
  
  # 回傳前先reshape調整至適合的維度
  return x_train_all.reshape(int(x_train_all.shape[0]/108), 108), x_train_pm25.reshape(int(x_train_pm25.shape[0]/6), 6), np.array(df_y, dtype=float).reshape(df_y.shape[0], 1)

In [ ]:
def evaluation_MAE(x_train, y_train, x_test, y_test, hr, label): # hr為預測的目標小時 , label用到的feature數
  def linear_reg(x_train, y_train, x_test, y_test):
    reg = LinearRegression(normalize=True, n_jobs=-1).fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    print("使用LinearRegressior做為預測模型")
    # print("Acc : {}".format(reg.score(x_train, y_train).round(4)))
    if label == 1:
      print("只用 PM2.5 Label 預測未來{0}小時的MAE : {1}\n".format(hr, mean_absolute_error(y_test, y_pred)))
    else:
      print("使用 全部的Label 預測未來{0}小時的MAE : {1}\n".format(hr, mean_absolute_error(y_test, y_pred)))
    

  def xgboost_reg(x_train,y_train, x_test, y_test):
    reg = XGBRegressor(objective='reg:squarederror', n_jobs=-1).fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    print("使用XGBRegressor做為預測模型")
    #　print("Acc : {}".format(reg.score(x_train, y_train).round(4)))
    if label == 1:
      print("使用 PM 2.5 Label 預測未來{0}小時的MAE : {1} \n".format(hr, mean_absolute_error(y_test, y_pred)))
    else:
      print("使用 全部的Label 預測未來{0}小時的MAE : {1} \n".format(hr, mean_absolute_error(y_test, y_pred)))
 
  linear_reg(x_train, y_train, x_test, y_test)
  xgboost_reg(x_train, y_train, x_test, y_test)

In [ ]:
df = pd.read_csv('新竹_2020.csv', encoding='big5')

# 資料清理
star_symbol = df.iloc[168, 23] # 取出星號的編碼值

check = ['#', 'x', 'A']
for x in check:
  df.iloc[:, 3:]= df.iloc[:, 3:].replace(x, -1, regex=True) 

df = df.replace(star_symbol, -1) # 取代星號
df = df.iloc[-1656:, :].reset_index().iloc[:, 2:] # 切割10,11,12月資料
df.iloc[:, 2:] = df.iloc[:, 2:].astype('float')
df.iloc[1650, 11:] = 3.75 # 最後一天的NO2的缺值，由於沒有1/1的資料所以取該天平均 
fill(df)
df = df.iloc[:, 1:] # 將觀測站和日期欄拿掉
df

In [ ]:
T_df = df.iloc[0:18, 0:25].reset_index().T.iloc[1:, :]

# 做矩陣轉置
for i in range(18, 1655, 18):
  T_df = pd.concat([T_df, df.iloc[i:i+18, 1:25].reset_index().T.iloc[1:, :]], ignore_index=True) # 轉置後合併

T_df

In [ ]:
train = T_df.iloc[1:1465, :] # 第一列為col inedx去除 10~11月資料
test = T_df.iloc[1465:, :] # 12月資料

x_train_all_feature_hr1, x_train_pm25_hr1, x_test_hr1 = split_dataset(train, T_df.iloc[7:1465, 9], 6)
y_train_all_feature_hr1, y_train_pm25_hr1, y_test_hr1 = split_dataset(test, T_df.iloc[1465+6:, 9], 6)
x_train_all_feature_hr6, x_train_pm25_hr6, x_test_hr6 = split_dataset(train, T_df.iloc[12:1465, 9], 11)
y_train_all_feature_hr6, y_train_pm25_hr6, y_test_hr6 = split_dataset(test, T_df.iloc[1465+11:, 9], 11)


# 確認資料是否正確用
# pd.DataFrame(y_train_all_feature_hr6)
# pd.DataFrame(y_train_pm25_hr6)
# pd.DataFrame(y_test_hr6)
# x_train_all_feature_hr1.shape
# x_test_hr1.shape


In [ ]:
evaluation_MAE(x_train_pm25_hr1, x_test_hr1, y_train_pm25_hr1, y_test_hr1, 1, 1)
evaluation_MAE(x_train_all_feature_hr1, x_test_hr1, y_train_all_feature_hr1, y_test_hr1, 1, 0)
evaluation_MAE(x_train_pm25_hr6, x_test_hr6, y_train_pm25_hr6, y_test_hr6, 6, 1)
evaluation_MAE(x_train_all_feature_hr6, x_test_hr6, y_train_all_feature_hr6, y_test_hr6, 6, 0)